<a href="https://colab.research.google.com/github/nonoumasy/LA-County-Covid-Cases/blob/master/LA_County_Covid_Cases_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from geopy.geocoders import Nominatim

In [0]:
# load data from url
df = pd.DataFrame(pd.read_html('http://publichealth.lacounty.gov/media/Coronavirus/locations.htm')[0])

In [0]:
# skip rows
new_df = df.iloc[22:, :].reset_index(drop=True)

new_df.columns = df.columns.str.lower().str.replace(' ', '_')

new_df = new_df[new_df.locations != 'City of Los Angeles']

# cleanup values
new_df.locations = new_df.locations.str.replace('*', '').str.replace('City of ', '').str.replace('Los Angeles - ', '').str.replace('Unincorporated - ', '')
new_df['total_cases'] = new_df.total_cases.str.replace('--', '0')

# convert to numeric column
new_df['total_cases'] = pd.to_numeric(new_df['total_cases'])

# drop under investigation
new_df = new_df[new_df.locations != '- Under Investigation']

# split location and keep first value
new_df['locations'] = new_df['locations'].apply(lambda x : x.split('/')[0]).apply(lambda x : x.split('-')[0])

#
new_df = new_df.sort_values('locations')

# reset index
new_df.reset_index(drop=True, inplace=True)

In [0]:
# load locations_df with geocoded
locations_df = pd.read_csv('/content/locations_df.csv')

# cleanup
locations_df = locations_df[['Locations', 'lat', 'lon']]
locations_df.columns= locations_df.columns = ['locations', 'lat', 'lon']

# merge with new_df
update_df = pd.merge(new_df, locations_df, how='inner', left_on='locations', right_on='locations')

# 
update_df.to_csv('la_county_coronavirus.csv')

In [0]:
# prep location for geocoding. Explicitly add ', Los Angeles, California' so the geocoder finds the location
new_df['loc_prep'] = 'temp'
for i in range(len(new_df)):
  new_df['loc_prep'][i] = str(new_df.Locations[i]) + ', Los Angeles, California'

# create and initialize coords column
new_df['coords'] = 0.0

# geocode locations
geolocator = Nominatim(user_agent="sdfasdf")

# to space out the geocoding
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
new_df['coords'] = new_df.loc_prep.apply(lambda x : geocode(x))

#
new_df['lat'] = new_df.coords.apply(lambda x: x.latitude if x !=None else 0.0)
new_df['lon'] = new_df.coords.apply(lambda x: x.longitude if x !=None else 0.0)

# save to csv
new_df.to_csv('la_county_coronavirus.csv')

# save death table
age_df = pd.DataFrame((df.iloc[12:16, :]))
age_df.columns = ['age_group', 'cases']
age_df.to_csv('age_df.csv')

# 
new_df.to_csv('la_county_covid.csv')

# save locations(geocoded) to a csv
locations_df = new_df[['Locations', 'lat', 'lon']]
locations_df.to_csv('locations_df.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
new_df.head(2)

,locations,total_cases
0,Acton,0
1,Adams,0
